In [ ]:
import talib
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [ ]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [ ]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'test1.csv')
# df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [ ]:
def add_fractal(df, i, look_back=3):
  if i >= look_back and i < df.shape[0] - look_back:
    lows = []
    highs = []

    for j in range(1, look_back + 1):
      prev_bid_low, prev_bid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
      future_bid_low, future_bid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

      lows.append(float(prev_bid_low))
      lows.append(float(future_bid_low))
      highs.append(float(prev_bid_high))
      highs.append(float(future_bid_high))

    bid_low, bid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

    if float(bid_low) < min(lows):
      return 1

    elif float(bid_high) > max(highs):
      return 2

    else:
      return 0

  else:
    return np.nan


def add_beep_boop(df, i):
  macdhist, ema50, ema200, bid_low, bid_high = df.loc[df.index[i], ['macdhist', 'ema50', 'ema200', 'Mid_Low', 'Mid_High']]

  if float(macdhist) > 0 and float(bid_low) > float(ema50):
    return 1

  elif float(macdhist) < 0 and float(bid_high) < float(ema50):
    return 2

  else:
    return 0


def add_trade(df, i, atr_percentage=0.3):
    if i < 3:
        return np.nan

    beep_boop1 = df.loc[df.index[i - 1], 'beep_boop']
    beep_boop2 = df.loc[df.index[i - 2], 'beep_boop']
    beep_boop3 = df.loc[df.index[i - 3], 'beep_boop']
    atr = df.loc[df.index[i - 1], 'atr']
    curr_ao = df.loc[df.index[i], 'Ask_Open']
    curr_bo = df.loc[df.index[i], 'Bid_Open']
    spread = abs(curr_ao - curr_bo)
    enough_volatility = (spread / atr) <= atr_percentage

    if beep_boop3 == 1 and beep_boop2 == 1 and beep_boop1 == 1 and enough_volatility:
        return 1

    elif beep_boop3 == 2 and beep_boop2 == 2 and beep_boop1 == 2 and enough_volatility:
        return 2

    else:
        return 0

In [ ]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
df['atr'] = talib.ATR(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=500)
df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df['beep_boop'] = [add_beep_boop(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df['trade'] = [add_trade(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['beep_boop'].value_counts())
print(df['fractal'].value_counts())
print(df['trade'].value_counts())

In [ ]:
# df_fractals = df[df.fractal != 0]
# df_fractal_prices = [
#     df_fractals.loc[df_fractals.index[i], 'Mid_Low'] if df_fractals.loc[df_fractals.index[i], 'fractal'] == 1 else df_fractals.loc[
#         df_fractals.index[i], 'Mid_High'] for i in range(len(df_fractals))]

In [ ]:
def plot_candles(df_plot):
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df_plot.Date, open=df_plot.Mid_Open, high=df_plot.Mid_High, low=df_plot.Mid_Low, close=df_plot.Mid_Close,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',
        decreasing_line_color='#FF3A4C'
    ))

    df_fractals = df_plot[df_plot.fractal != 0]
    df_fractal_prices = [df_fractals.loc[df_fractals.index[i], 'Mid_Low'] if df_fractals.loc[df_fractals.index[i], 'fractal'] == 1 else df_fractals.loc[df_fractals.index[i], 'Mid_High'] for i in range(len(df_fractals))]

    fig.add_trace(go.Scatter(
        x=df_fractals.Date,
        y=df_fractal_prices,
        marker=dict(color="crimson", size=10),
        name='Fractal',
        mode='markers'
        ))

    df_buys = df_plot[df_plot.trade == 1]
    df_sells = df_plot[df_plot.trade == 2]

    fig.add_trace(go.Candlestick(
        x=df_buys.Date, open=df_buys.Mid_Open, high=df_buys.Mid_High, low=df_buys.Mid_Low, close=df_buys.Mid_Close,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='blue',
        decreasing_fillcolor='blue',
        increasing_line_color='blue',
        decreasing_line_color='blue',
        name='Buy'
    ))

    fig.add_trace(go.Candlestick(
        x=df_sells.Date, open=df_sells.Mid_Open, high=df_sells.Mid_High, low=df_sells.Mid_Low, close=df_sells.Mid_Close,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='orange',
        decreasing_fillcolor='orange',
        increasing_line_color='orange',
        decreasing_line_color='orange',
        name='Sell'
    ))

    fig.update_layout(width=1400,height=600,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")

    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )

    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )

    fig.show()

In [ ]:
plot_candles(df.iloc[47100:47300, :])